In [7]:
# %%writefile vote_examiner.py
from pymongo import MongoClient
import bson.json_util
import os
import pandas as pd
import numpy as np
import pandas as pd
import boto
from boto.s3.connection import S3Connection, Location
from boto.s3.key import Key


from my_tools import read_jsonl_file


In [16]:
# create an AWS S3 connection
conn = boto.s3.connect_to_region('us-west-2', host = 's3-us-west-2.amazonaws.com')

legislation_bucket = conn.get_bucket('galvcap-leg')

print('Pulling {} files from S3...'.format('vote_results'))
print('-------------------')
for k in legislation_bucket:
    filename = k.key
    if filename.startswith('vote_results'):
        k.get_contents_to_filename('../data/{}'.format(filename))


Pulling vote_results files from S3...
-------------------
vote_results_1990.jsonl
vote_results_1991.jsonl
vote_results_1992.jsonl
vote_results_1993.jsonl
vote_results_1994.jsonl
vote_results_1995.jsonl
vote_results_1996.jsonl
vote_results_1997.jsonl
vote_results_1998.jsonl
vote_results_1999.jsonl
vote_results_2000.jsonl
vote_results_2001.jsonl
vote_results_2002.jsonl
vote_results_2003.jsonl
vote_results_2004.jsonl
vote_results_2005.jsonl
vote_results_2006.jsonl
vote_results_2007.jsonl
vote_results_2008.jsonl
vote_results_2009.jsonl
vote_results_2010.jsonl
vote_results_2011.jsonl
vote_results_2012.jsonl
vote_results_2013.jsonl
vote_results_2014.jsonl
vote_results_2015.jsonl
vote_results_2016.jsonl
vote_results_2017.jsonl
vote_results_2018.jsonl


In [18]:
# get file data into dataframe
data = pd.DataFrame()
for filename in os.listdir('../data'):
    if filename.startswith('vote_results'):
        vote_results = read_jsonl_file('../data/{}'.format(filename))
        data = pd.concat([data, pd.DataFrame(list(vote_results))])

In [44]:
data.head()

,date,description,issue,question,result,roll,vote_results,year,issue_type
0,26-Feb,Huffman of California Amendment No. 16,H R 2406,On Agreeing to the Amendment,F,99,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
1,26-Feb,Young of Alaska Amendment No. 15,H R 2406,On Agreeing to the Amendment,A,98,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
2,26-Feb,Ribble of Wisconsin Amendment No. 14,H R 2406,On Agreeing to the Amendment,A,97,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
3,26-Feb,Griffith of Virginia Amendment No. 12,H R 2406,On Agreeing to the Amendment,A,96,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
4,26-Feb,Smith of Missouri Amendment No. 9,H R 2406,On Agreeing to the Amendment,A,95,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R


In [88]:
# Result designators are P for Passed, F for Failed, and A for Agreed To
data['result'].value_counts()

F    5470
P    5247
A    2111
Name: result, dtype: int64

In [43]:
# reduce dataframe to just bills and joint resolutions by first creating a column
# for issue_type and then filtering on it
data['issue_type'] = data['issue'].apply(lambda x: x.rsplit(' ', 1)[0])

data = data[(data['issue_type'] == 'H R') | 
            (data['issue_type'] == 'H J RES') | 
            (data['issue_type'] == 'S') | 
            (data['issue_type'] == 'S J')]

In [100]:
data.question.value_counts()[:10]

On Agreeing to the Amendment                           5992
On Passage                                             1928
On Motion to Suspend the Rules and Pass                1092
On Motion to Suspend the Rules and Pass, as Amended    1013
On Motion to Recommit with Instructions                 876
On Agreeing to the Conference Report                    360
On Motion to Instruct Conferees                         222
Suspend the rules and pass, as amended                   80
SUSPEND THE RULES AND PASS, AS AMENDED                   69
On Motion that the Committee Rise                        62
Name: question, dtype: int64

In [101]:
# examine just the roll call votes with 'pass' in question]
passed = data[(data['question'].str.contains('pass')) | 
              (data['question'].str.contains('Pass')) | 
              (data['question'].str.contains('PASS'))]


In [102]:
passed.head()

,date,description,issue,question,result,roll,vote_results,year,issue_type
10,25-Feb,Fraudulent Joinder Prevention Act,H R 3624,On Passage,P,89,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
15,23-Feb,Foreign Fighter Review Act of 2016,H R 4402,"On Motion to Suspend the Rules and Pass, as Am...",P,84,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
16,23-Feb,National Strategy to Combat Terrorist Travel A...,H R 4408,"On Motion to Suspend the Rules and Pass, as Am...",P,83,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
18,12-Feb,Common Sense Nutrition Disclosure Act of 2015,H R 2017,On Passage,P,81,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R
23,11-Feb,Debt Management and Fiscal Responsibility Act ...,H R 3442,On Passage,P,76,"[{'name_id': None, 'name': 'Abraham', 'party':...",2016,H R


In [96]:
grouped_data[(grouped_data['issue'] == 'H R 2406') & (grouped_data['year'] == 2016)]

,issue,year,question,result,count
2425,H R 2406,2016,On Agreeing to the Amendment,A,4
2426,H R 2406,2016,On Agreeing to the Amendment,F,4
2427,H R 2406,2016,On Motion to Recommit with Instructions,F,1
2428,H R 2406,2016,On Passage,P,1


In [91]:
# group the data to get a count of question types for each bill
grouped_data = data.groupby(['issue', 'year', 'question', 'result']).count()
grouped_data = pd.DataFrame(grouped_data.iloc[:, 0])

# dataframe is multiindexed after groupby. Reset index and rename column
grouped_data.reset_index(inplace=True)
grouped_data = grouped_data.rename(columns = {'date': 'count'})

In [93]:
grouped_data.head(20)

,issue,year,question,result,count
0,H J RES 1,1995,On Agreeing to the Amendment,A,2
1,H J RES 1,1995,On Agreeing to the Amendment,F,4
2,H J RES 1,1995,On Motion to Recommit with Instructions,F,1
3,H J RES 1,1995,On Passage,P,1
4,H J RES 1,2003,On Motion to Recommit with Instructions,F,1
5,H J RES 1,2003,Table the appeal of the ruling of the Chair,P,1
6,H J RES 10,2005,On Agreeing to the Amendment,F,1
7,H J RES 10,2005,On Passage,P,1
8,H J RES 10,2005,Table Appeal of the Ruling of the Chair,P,2
9,H J RES 100,2000,On Motion to Suspend the Rules and Pass,P,1


In [94]:
pass.question.unique()

array(['On Agreeing to the Amendment',
       'On Motion to Recommit with Instructions', 'On Passage',
       'Table the appeal of the ruling of the Chair',
       'Table Appeal of the Ruling of the Chair',
       'On Motion to Suspend the Rules and Pass',
       'On Motion to Suspend the Rules and Pass, as Amended',
       'On Agreeing to the Resolution',
       'On Motion to Dispose of Senate Amendments En Bloc',
       'Postpone consideration of Veto Message to a date certain',
       'On motion to agree to the Senate Amendment',
       'Table the Appeal of the Ruling of the Chair',
       'On Agreeing to the Conference Report',
       'Suspend the rules and pass, as amended',
       'On Motion to Instruct Conferees',
       'Recommit Conference Report with Instructions',
       'On consideration of the joint resolution',
       'Table Motion to Reconsider',
       'On Consideration of the Joint Resolution',
       'SUSPEND THE RULES AND PASS AS AMENDED',
       'On Adoption of the 

In [54]:
    if filename.startswith('vote_results'):
        k.get_contents_to_filename('../data/{}'.format('vote_results'))

In [2]:
# intialize mongo client
client = MongoClient() # defaults to localhost
db = client.bills
vote_results = db.vote_results

In [3]:
vote_results.find().count()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


18570

In [4]:
all_results = vote_results.find()

In [5]:
all_results[0]

{'_id': ObjectId('5c1bd1cd1417de0d83d6181d'),
 'year': 2016,
 'roll': '99',
 'date': '26-Feb',
 'issue': 'H R 2406',
 'question': 'On Agreeing to the Amendment',
 'result': 'F',
 'description': 'Huffman of California Amendment No. 16',
 'vote_results': [{'name_id': None,
   'name': 'Abraham',
   'party': 'R',
   'state': 'LA',
   'vote': 'No'},
  {'name_id': None,
   'name': 'Adams',
   'party': 'D',
   'state': 'NC',
   'vote': 'Aye'},
  {'name_id': None,
   'name': 'Aderholt',
   'party': 'R',
   'state': 'AL',
   'vote': 'No'},
  {'name_id': None,
   'name': 'Aguilar',
   'party': 'D',
   'state': 'CA',
   'vote': 'Aye'},
  {'name_id': None,
   'name': 'Allen',
   'party': 'R',
   'state': 'GA',
   'vote': 'No'},
  {'name_id': None,
   'name': 'Amash',
   'party': 'R',
   'state': 'MI',
   'vote': 'No'},
  {'name_id': None,
   'name': 'Amodei',
   'party': 'R',
   'state': 'NV',
   'vote': 'Not Voting'},
  {'name_id': None,
   'name': 'Ashford',
   'party': 'D',
   'state': 'NE',
  

In [6]:
all_results[0].keys()

dict_keys(['_id', 'year', 'roll', 'date', 'issue', 'question', 'result', 'description', 'vote_results'])

In [ ]:
pd.DataFrame(all_results[0])

In [ ]:
i = 0
data = pd.DataFrame()
for result in all_results:
    print('record: {}'.format(i))
    
    data = pd.concat([data, pd.DataFrame(result)])

In [ ]:
# if i build a function here, do i want a date parameter?
data = pd.DataFrame()
for filename in os.listdir('../data'):
    if filename.startswith('vote_results'):
        print(filename)
        vote_results = read_json_lines_file('../data/{}'.format(filename))
        data = pd.concat([data, pd.DataFrame(list(vote_results))])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
print('--------------------------')
print('--------------------------')
print('Top 20 questions during a voting session:')
print('--------------------------')
print(data.question.value_counts()[:20])


In [ ]:
# create a dataframe for recent years
recent_df = data[(data['year'] == 2018) | 
                 (data['year'] == 2017) | 
                 (data['year'] == 2016) | 
                 (data['year'] == 2015) | 
                 (data['year'] == 2014) | 
                 (data['year'] == 2013) | 
                 (data['year'] == 2012) | 
                 (data['year'] == 2011) | 
                 (data['year'] == 2010) | 
                 (data['year'] == 2009) |
                 (data['year'] == 2008) | 
                 (data['year'] == 2007)
                ]

In [ ]:
recent_df.shape

In [ ]:
recent_df.head()

In [ ]:
congress_dict = {
    2007: '110', 
    2008: '110', 
    2009: '111', 
    2010: '111', 
    2011: '112', 
    2012: '112', 
    2013: '113', 
    2014: '113', 
    2015: '114', 
    2016: '114', 
    2017: '115', 
    2018: '116' 
    
}

In [ ]:
for i in recent_df.year.unique():
    print(type(i))

In [ ]:
np.array(recent_df['year'])

In [ ]:
recent_df['congress_id'] = None

In [ ]:
recent_df.replace({'congress_id': congress_dict})

In [ ]:
recent_df.congress_id.unique()